In [ ]:
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os librosa

from tensorflow.keras.layers import Dense, LSTM, Input, TimeDistributed, Bidirectional, Activation
from tensorflow.keras.regularizers import l2
from tensorflow.keras.models import Model

In [ ]:
def onehot(str):
    path = './Drum'
    kits = os.listdir(path)
    #kits.remove('.DS_Store')
    
    oh = []

    for kit in kits:
        if str == kit:
            oh.append(1)
        else:
            oh.append(0)
    
    return np.array(oh)

In [ ]:
def make4096(wav):
    while len(wav) < 4096:
        wav = np.append(wav, np.array([0]))
    return wav[:4096]
            

---

In [ ]:
#path = 'Drum\Snare\9th Snare 38.wav'

y_label = None
x_label = None

drumkit_path = './Drum'
kits = os.listdir(drumkit_path)
n=0

for kit in kits:
    path = os.path.join(drumkit_path, kit)
    sounds = os.listdir(path)
    
    for sound in sounds:
        wavfile = os.path.join(path, sound)
        y, sr = librosa.load(wavfile)
        yt, index = librosa.effects.trim(y=y, top_db=30)
        y_16k = librosa.resample(yt, orig_sr=sr, target_sr=16000)
        #yt16k = librosa.util.normalize(y_16k)
        n += len(y_16k)
        yt16k = make4096(y_16k)
        yt16k = np.expand_dims(yt16k, axis=0)
        
        typ = onehot(kit)
        typ = np.expand_dims(typ, axis=0)
        
        if x_label is None:
            x_label = yt16k.copy()
            y_label = typ.copy()
        else:
            x_label = np.concatenate((x_label, yt16k), axis=0)
            y_label = np.concatenate((y_label, typ), axis=0)
        
        

In [ ]:
n / 700

In [ ]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(x_label, y_label, test_size=0.2, random_state=1234, stratify=y_label)

In [ ]:
yy_test = np.argmax(y_test, axis = 1)
yy_train = np.argmax(y_train, axis = 1)

---

In [ ]:
units = 128
dropout = 0.2

model = tf.keras.models.Sequential()
forward_layer = tf.keras.layers.LSTM(units=units, dropout=dropout, recurrent_dropout=dropout)
backward_layer = tf.keras.layers.LSTM(units=units, activation='relu', go_backwards=True, dropout=dropout, recurrent_dropout=dropout)
model.add(Bidirectional(forward_layer, backward_layer=backward_layer, input_shape=(4096)))
model.add(Dense(7, activation='softmax'))

model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])


In [ ]:
history = model.fit(x_train, yy_train, epochs=20, validation_split=0.2, verbose = 0)

In [ ]:
import matplotlib.pyplot as plt

acc = history.history['loss']
val_acc = history.history['val_loss']

loss = history.history['accuracy']
val_loss = history.history['val_accuracy']

epochs_range = range(20)

plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Loss')
plt.plot(epochs_range, val_acc, label='Validation Loss')
plt.legend(loc='lower right')
plt.title('Training and Validation Loss')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Acc')
plt.plot(epochs_range, val_loss, label='Validation Acc')
plt.legend(loc='upper right')
plt.title('Training and Validation Acc')
plt.show()

In [ ]:
y_pred =model.predict(x_test)
ypred = np.argmax(y_pred,axis=1)

In [ ]:
model.evaluate(x_test,yy_test)

In [ ]:
from sklearn.metrics import confusion_matrix
import pandas as pd
import seaborn as sns

classes = ['Conga&Bongo', 'Crash&OpenHat', 'Hi-Hat', 'Kick', 'Shaker', 'Snare', 'Tom']

cm = pd.DataFrame(confusion_matrix(yy_test, ypred), columns=classes, index=classes)
sns.heatmap(cm, annot=True)